In [7]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fairbet_django.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [8]:
from order_placed.models import *
from django_pandas.io import read_frame

In [9]:
df = read_frame(Betting.objects.all())

In [10]:
df

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at
0,4,fairbet,200.0,Srilanka,BACK,1.85,Not declared yet,0.0,Srilanka VS India,2022-07-04 06:21:06.148020+00:00,2022-07-04 06:21:06.158295+00:00
1,5,fairbet,200.0,Srilanka,BACK,1.85,Not declared yet,0.0,Srilanka VS India,2022-07-04 06:21:17.505599+00:00,2022-07-04 06:21:17.509525+00:00
2,6,fairbet,100.0,Srilanka,BACK,1.85,Not declared yet,0.0,Srilanka VS India,2022-07-04 06:21:30.086292+00:00,2022-07-04 06:21:30.092277+00:00
